In [2]:
# https://hackmd.io/@s02260441/SkA7IWVJv 代码地址
# pip install 


Note: you may need to restart the kernel to use updated packages.


In [1]:
import yfinance as yf
import pandas as pd
from pandas_datareader import data
from datetime import datetime

yf.pdr_override() #以pandasreader常用的格式覆寫

target_stock = 'TSLA'  #股票代號變數

start_date = datetime(2010, 1, 1)
end_date = datetime(2020, 6, 30) #設定資料起訖日期

df = data.get_data_yahoo([target_stock], start_date, end_date) #將資料放到Dataframe裡面

print(df)
filename = f'./data/{target_stock}.csv' #以股票名稱命名檔案，放在data資料夾下面

df.to_csv(filename) #將df轉成CSV保存

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2010-06-29    3.800000    5.000000    3.508000    4.778000    4.778000   
2010-06-30    5.158000    6.084000    4.660000    4.766000    4.766000   
2010-07-01    5.000000    5.184000    4.054000    4.392000    4.392000   
2010-07-02    4.600000    4.620000    3.742000    3.840000    3.840000   
2010-07-06    4.000000    4.000000    3.166000    3.222000    3.222000   
...                ...         ...         ...         ...         ...   
2020-06-23  199.776001  202.399994  198.802002  200.356003  200.356003   
2020-06-24  198.822006  200.175995  190.628006  192.169998  192.169998   
2020-06-25  190.854004  197.195999  187.429993  197.195999  197.195999   
2020-06-26  198.955994  199.000000  190.973999  191.947998  191.947998   
2020-06-29  193.802002  202.000000  189.703

In [3]:
from backtesting import Backtest, Strategy #引入回測和交易策略功能

from backtesting.lib import crossover #從lib子模組引入判斷均線交會功能
from backtesting.test import SMA #從test子模組引入繪製均線功能

import pandas as pd #引入pandas讀取股價歷史資料CSV檔

/usr/local/lib/python3.7/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [4]:
class SmaCross(Strategy): #交易策略命名為SmaClass，使用backtesting.py的Strategy功能
    n1 = 5 #設定第一條均線日數為5日(周線)
    n2 = 20 #設定第二條均線日數為20日(月線)，這邊的日數可自由調整

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1) #定義第一條均線為sma1，使用backtesting.py的SMA功能算繪
        self.sma2 = self.I(SMA, self.data.Close, self.n2) #定義第二條均線為sma2，使用backtesting.py的SMA功能算繪

    def next(self):
        if crossover(self.sma1, self.sma2): #如果周線衝上月線，表示近期是上漲的，則買入
            self.buy()
        elif crossover(self.sma2, self.sma1): #如果周線再與月線交叉，表示開始下跌了，則賣出
            self.sell()

In [5]:
stock = "TSLA" #設定要測試的股票標的名稱

df = pd.read_csv(f"./data/{stock}.csv", index_col=0) #pandas讀取資料，並將第1欄作為索引欄
df = df.interpolate() #CSV檔案中若有缺漏，會使用內插法自動補值，不一定需要的功能
df.index = pd.to_datetime(df.index) #將索引欄資料轉換成pandas的時間格式，backtesting才有辦法排序

In [6]:
test = Backtest(df, SmaCross, cash=10000, commission=.002)
# 指定回測程式為test，在Backtest函數中依序放入(資料來源、策略、現金、手續費)

result = test.run()
#執行回測程式並存到result中

In [7]:
print(result) # 直接print文字結果

test.plot(filename=f"./backtest_result/{stock}.html") #將線圖網頁依照指定檔名保存

Start                     2010-06-29 00:00:00
End                       2020-06-29 00:00:00
Duration                   3653 days 00:00:00
Exposure Time [%]                     16.2828
Equity Final [$]                            0
Equity Peak [$]                       10564.2
Return [%]                               -100
Buy & Hold Return [%]                 4124.99
Return (Ann.) [%]                           0
Volatility (Ann.) [%]                  1523.2
Sharpe Ratio                                0
Sortino Ratio                               0
Calmar Ratio                                0
Max. Drawdown [%]                        -100
Avg. Drawdown [%]                        -100
Max. Drawdown Duration     3609 days 00:00:00
Avg. Drawdown Duration     3609 days 00:00:00
# Trades                                    1
Win Rate [%]                                0
Best Trade [%]                       -100.508
Worst Trade [%]                      -100.508
Avg. Trade [%]                    

Row(id='1450', ...)